# Policy Gradient

Originally from https://skettee.github.io/post/policy_gradient/ (in Korean)

## Load Libraries and Extensions

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import random
from collections import deque
from IPython.display import display, clear_output, Pretty
import numpy as np
import os
from pprint import pprint
from time import sleep
from tqdm import tqdm_notebook as tqdm

import gym

## Lunar Lander Environment

In [3]:
ENV_NAME = 'LunarLanderContinuous-v2'
N_STEP = 100

In [4]:
env = gym.make(ENV_NAME)
state = env.reset()
action = env.action_space.sample()

print('State space: ', env.observation_space)
print('Initial state: ', state)
print('\nAction space: ', env.action_space)
print('Random action: ', action)

State space:  Box(8,)
Initial state:  [-0.00424175  1.4180917  -0.42966294  0.31872183  0.00492195  0.09732513
  0.          0.        ]

Action space:  Box(2,)
Random action:  [-0.9385834   0.45807928]


### Action

Each action is defined as 2 real numbers between -1 and 1.

$A = \begin{bmatrix}
\{ a_{00}, a_{01} \}, \\
\{ a_{10}, a_{11} \}, \\
\{ a_{20}, a_{21} \}, \\
\{ a_{30}, a_{31} \}
\end{bmatrix}$  

Index | State                   | Control
------|-----------------|-------------
0     | Main engine       | -1..0 off, 0..+1 throttle from 50% to 100% power
1     | Left or right engine | -1.0..-0.5 fire left engine, +0.5..+1.0 fire right engine, -0.5..0.5 off


### State

$S = \begin{bmatrix}
\{ s_{00}, s_{01}, \cdots\}, \\
\{ s_{10}, s_{11}, \cdots \}, \\
\{ s_{20}, s_{21}, \cdots \}, \\
\{ s_{30}, s_{31}, \cdots \}
\end{bmatrix}$  

Index | State                                           | Value
-----|---------------------------------|-----------
0      | X position               | -Inf ~ Inf
1      | Y position               | -Inf ~ Inf
2      | X velocity                                | -Inf ~ Inf
3      | Y velocity                                | -Inf ~ Inf
4      | Angle                  | -Inf ~ Inf
5      | Angular velocity | -Inf ~ Inf
6      | If the left lag touches the ground           | 1 (Yes), 0 (No)
7      | If the right lag touches the ground         | 1 (Yes), 0 (No)


### Reward

- Reward for moving from the top of the screen to landing pad and zero speed is about 100..140 points.
- If lander moves away from landing pad it loses reward back.
- Episode finishes if the lander crashes or comes to rest, receiving additional -100 or +100 points.
- Each leg ground contact is +10
- Firing main engine is -0.3 points each frame
- Firing side engine is -0.03 points each frame
- Solved is 200 points

## Policy Gradient Model

$X = \{ s_1^{(i)}, s_2^{(i)}, s_3^{(i)}, s_4^{(i)},  s_5^{(i)}, s_6^{(i)}, s_7^{(i)}, s_8^{(i)} \}$

$Y = \{a_1^{(i)}, a_2^{(i)})\}$

$\begin{align}
& \hat Y = \{ \hat a_1^{(i)}, \hat a_2^{(i)} \} \\
& \hat a_1^{(i)} = \hat \mu_1 (s, \theta) \\
& \hat a_2^{(i)} = \hat \mu_2 (s, \theta)
\end{align}$

$J(\theta) = - \dfrac{1}{N} \sum_{i=1}^N  \nabla_{\theta} \log p(a ; \theta) * \hat A_{\text{std}}$


$\theta \leftarrow \theta-\alpha \dfrac{\partial J(\theta)}{\partial \theta}$

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Dense

# Input Dimensions
states_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]

# Placeholders for inputs.
input_state = tf.placeholder(shape=[None, states_dim], name="state", dtype=tf.float32)
input_action = tf.placeholder(shape=[None, action_dim], name="action", dtype=tf.float32)
input_advan = tf.placeholder(shape=[None], name="advan", dtype=tf.float32)
n = tf.shape(input_state)[0]

# The log std vector, it's a parameter.
logstd = tf.get_variable("logstd", shape=[action_dim], initializer=tf.zeros_initializer())
logstd_n = tf.ones(shape=(n, action_dim), dtype=tf.float32) * logstd

# The policy network.
hidden1 = Dense(32, activation='relu')(input_state)
hidden2 = Dense(32, activation='relu')(hidden1)
mean = Dense(action_dim, activation=None)(hidden2)

# Computing the log likelihood. 
variance = tf.exp(2*logstd_n)
func = -tf.square(input_action - mean)/(2*variance) - 0.5*tf.log(tf.constant(2*np.pi)) - logstd_n
log_likelihood = tf.reduce_sum(func, axis=[1])

# Diagonal Gaussian distribution for sampling actions
sampled_action = (tf.random_normal(tf.shape(mean)) * tf.exp(logstd_n) + mean)[0]

# Loss function
loss_op = - tf.reduce_mean(log_likelihood * input_advan) 
# Optimizer
update = tf.train.AdamOptimizer().minimize(loss_op) 

W1029 20:23:45.812487 4598023616 deprecation.py:506] From /Users/jeong/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## State-Value Model

$X = \{ s_1^{(i)}, s_2^{(i)}, s_3^{(i)}, s_4^{(i)},  s_5^{(i)}, s_6^{(i)}, s_7^{(i)}, s_8^{(i)}\}$

$Y = v^{(i)} = G^{(i)}_t$

$\hat Y = \hat v^{(i)} =  w^T X + b $

${\large J}(w, b) = \dfrac{1}{2m} \sum_{i=1}^m (G_t^{(i)} - \hat v^{(i)} (S, w, b))^2$  

$\begin{align}
w & \leftarrow w-\alpha \dfrac{\partial J(w,b)}{\partial w} \\    
b & \leftarrow b-\alpha \dfrac{\partial J(w,b)}{\partial b}
\end{align}$  

In [6]:
from sklearn.linear_model import LinearRegression
# State-value function approximator
approx = LinearRegression()

# Initialization
approx.fit(np.array([0]*states_dim).reshape((1,states_dim)), np.array([0]).reshape((1,1)))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [7]:
import scipy.signal

def get_returns(x, gamma):
    return scipy.signal.lfilter([1],[1,-gamma],x[::-1], axis=0)[::-1]

def predict(sess, state):
    with sess.as_default():
        action = sess.run(sampled_action, feed_dict={input_state: state[None]})
    return action

def fit(sess, state, action, advantage):
    with sess.as_default():
        sess.run([update], feed_dict={input_state: state, input_action: action, input_advan: advantage})      

## Training

In [8]:
from tqdm import tqdm
from tensorflow.keras import backend as K

sess = tf.Session()
K.set_session(sess)
sess.run(tf.global_variables_initializer())

num_iteration = 500
gamma = 0.97
min_timesteps_per_batch = 3000
total_timesteps = 0

for i in tqdm(range(num_iteration)):
    timesteps_this_batch = 0
    memory = []
    while True:
        state = env.reset()
        states, actions, rewards = [], [], []
        while True:
            states.append(state)
            action = predict(sess, state)
            actions.append(action)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break
        # Memory
        path = {"state" : np.array(states), "action" : np.array(actions), "reward" : np.array(rewards)}
        memory.append(path)        
        timesteps_this_batch += len(path["reward"])
        if timesteps_this_batch > min_timesteps_per_batch:
            break
    total_timesteps += timesteps_this_batch
    
    # Replay
    
    # Advantage function
    G_ts, advs = [], []
    for path in memory:
        rew_t = path["reward"]
        G_t = get_returns(rew_t, gamma)
        v_hat = approx.predict(path["state"])
        adv = G_t - v_hat.flatten()
        advs.append(adv)
        G_ts.append(G_t)
   
    # Build data set
    in_states = np.concatenate([path["state"] for path in memory])
    in_actions  = np.concatenate([path["action"] for path in memory])
    advans = np.concatenate(advs)
    in_advans = (advans - advans.mean()) / (advans.std() + 1e-8)    
    G_ts = np.concatenate(G_ts)
    # Update State-value
    approx.fit(in_states, G_ts)
    # Update Policy
    fit(sess, in_states, in_actions, in_advans)

100%|██████████| 500/500 [34:20<00:00,  6.04s/it]


## Solution

In [10]:
env = gym.make(ENV_NAME)
state = env.reset()
done = False

while not done:
    env.render()

    action = predict(sess, state)
    state, reward, done, info = env.step(action)

env.close()